In [1]:
import keras
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

D:\LLAS\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


<h1>import Data</h1>

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./mnist/data/",one_hot=True)

Extracting ./mnist/data/train-images-idx3-ubyte.gz
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz


<h1>Set Hiper Piramiter</h1>

In [3]:
total_epoch = 200
batch_size = 100
learning_rate = 0.0001
n_hidden = 256
n_input = 28*28
n_noise = 128

<h1>Setting Input Tendor</h1>

In [4]:
#Real data
X = tf.placeholder(tf.float32,[None,n_input])
#Noise data
Z = tf.placeholder(tf.float32,[None,n_noise])
#Not used Y

<h1>Setting Tensor to Makes Hidden Layers</h1>

In [5]:
G_W1 = tf.Variable(tf.random_normal([n_noise, n_hidden],stddev=0.1))
G_B1 = tf.Variable(tf.zeros([n_hidden]))

G_W2 = tf.Variable(tf.random_normal([n_hidden, n_input],stddev=0.1))
G_B2 = tf.Variable(tf.zeros([n_input]))

<h1>Setting Tensor to Makes Discriminator Layers</h1>

In [6]:
D_W1 = tf.Variable(tf.random_normal([n_noise, n_hidden],stddev=0.1))
D_B1 = tf.Variable(tf.zeros([n_hidden]))

D_W2 = tf.Variable(tf.random_normal([n_hidden, n_input],stddev=0.1))
D_B2 = tf.Variable(tf.zeros([1]))

<h1>Create Generator</h1>

In [12]:
def generator(noise_z):
    hidden = tf.nn.relu( tf.matmul( noise_z, G_W1 ) + G_B1 )
    output = tf.nn.sigmoid( tf.matmul( hidden, G_W2 ) + G_B2 )
    return output

<h1>Create Discriminator</h1>

In [9]:
def discriminaster(inputs):
    hidden = tf.nn.relu( tf.matmul( inputs, D_W1 ) + D_B1 )
    output = tf.nn.sigmoid( tf.matmul( hidden, D_W2) + D_B2 )
    return output

<h1>Function to Make Noise-Data</h1>

In [10]:
def get_noise(batch_size, n_noise):
    return np.random.normal( size = ( batch_size, n_noise ) )

In [15]:
G = generator(Z)
D_gene  = discriminaster(G)
D_real = discriminaster(X)

ValueError: Dimensions must be equal, but are 784 and 128 for 'MatMul_7' (op: 'MatMul') with input shapes: [?,784], [128,256].